In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
%pylab inline

In [ ]:
ulist = pd.read_csv('data/user_list.csv')
ulist.info()

In [ ]:
cpltr = pd.read_csv('data/coupon_list_train.csv')
cpltr.info()

In [ ]:
cpdtr = pd.read_csv('data/coupon_detail_train.csv')
cpdtr.info()

In [ ]:
cpdtr[:3]

In [ ]:
cpvtr = pd.read_csv('data/coupon_visit_train.csv')
cpvtr.info()

In [ ]:
cpvtr[:3]

In [ ]:
cpvtr['COUPON_ID_hash'] = cpvtr['VIEW_COUPON_ID_hash']

In [ ]:
cplte = pd.read_csv('data/coupon_list_test.csv')
cplte.info()

In [ ]:
cplte[:5]

In [ ]:
train = pd.merge(cpdtr, cpltr, how='inner', on='COUPON_ID_hash', sort=True)
train = train[["COUPON_ID_hash","USER_ID_hash",
    "GENRE_NAME","DISCOUNT_PRICE","PRICE_RATE",
    "USABLE_DATE_MON","USABLE_DATE_TUE","USABLE_DATE_WED","USABLE_DATE_THU",
    "USABLE_DATE_FRI","USABLE_DATE_SAT","USABLE_DATE_SUN","USABLE_DATE_HOLIDAY",
    "USABLE_DATE_BEFORE_HOLIDAY","large_area_name","ken_name","small_area_name"]]
train.info()

In [ ]:
train[:3]

In [ ]:
train2 = pd.merge(cpvtr, cpltr, how='inner', on='COUPON_ID_hash', sort=True)
train2 = train2[["COUPON_ID_hash","USER_ID_hash",
    "GENRE_NAME","DISCOUNT_PRICE","PRICE_RATE",
    "USABLE_DATE_MON","USABLE_DATE_TUE","USABLE_DATE_WED","USABLE_DATE_THU",
    "USABLE_DATE_FRI","USABLE_DATE_SAT","USABLE_DATE_SUN","USABLE_DATE_HOLIDAY",
    "USABLE_DATE_BEFORE_HOLIDAY","large_area_name","ken_name","small_area_name"]]
train2.info()

In [ ]:
train2[:3]

In [ ]:
cplte['USER_ID_hash'] = 'dummyuser'
cpchar = cplte[["COUPON_ID_hash","USER_ID_hash",
    "GENRE_NAME","DISCOUNT_PRICE","PRICE_RATE",
    "USABLE_DATE_MON","USABLE_DATE_TUE","USABLE_DATE_WED","USABLE_DATE_THU",
    "USABLE_DATE_FRI","USABLE_DATE_SAT","USABLE_DATE_SUN","USABLE_DATE_HOLIDAY",
    "USABLE_DATE_BEFORE_HOLIDAY","large_area_name","ken_name","small_area_name"]]
cpchar[:3]

In [ ]:
cpchar.info()

In [ ]:
train = pd.concat([train, cpchar])
train.index = pd.Index(range(len(train)))
train.info()

In [ ]:
train2 = pd.concat([train2, cpchar])
train2.index = pd.Index(range(len(train2)))
train2.info()

In [ ]:
train = train.fillna(1)

In [ ]:
train2 = train2.fillna(1)

In [ ]:
from math import log10
from math import pow
import sys

In [ ]:
train.loc[train.DISCOUNT_PRICE <= 0, 'DISCOUNT_PRICE'] = 0.00001
train.DISCOUNT_PRICE = train.DISCOUNT_PRICE.apply(lambda x: 1. / log10(x))
train.PRICE_RATE = train.PRICE_RATE.apply(lambda x: pow(x, 2) / 10000.)

In [ ]:
train2.loc[train2.DISCOUNT_PRICE <= 0, 'DISCOUNT_PRICE'] = 0.00001
train2.DISCOUNT_PRICE = train2.DISCOUNT_PRICE.apply(lambda x: 1. / log10(x))
train2.PRICE_RATE = train2.PRICE_RATE.apply(lambda x: pow(x, 2) / 10000.)

In [ ]:
train[:3]

In [ ]:
train2[:3]

In [ ]:
train = pd.get_dummies(train, columns=['GENRE_NAME', 'large_area_name', 'ken_name', 'small_area_name'])

In [ ]:
train2 = pd.get_dummies(train2, columns=['GENRE_NAME', 'large_area_name', 'ken_name', 'small_area_name'])

In [ ]:
train.shape

In [ ]:
train2.shape

In [ ]:
train[:3]

In [ ]:
train2[:3]

In [ ]:
test = train.loc[train.USER_ID_hash == 'dummyuser']
test.info()

In [ ]:
test.columns

In [ ]:
# test <- test[,-2]
del test['USER_ID_hash']
test.info()

In [ ]:
test[:3]

In [ ]:
train = train.loc[train.USER_ID_hash != 'dummyuser']
train.info()

In [ ]:
train2 = train2.loc[train2.USER_ID_hash != 'dummyuser']
train2.info()

In [ ]:
train.columns

In [ ]:
train2.columns

In [ ]:
train[:3]

In [ ]:
train2[:3]

In [ ]:
del train['COUPON_ID_hash']
grouped = train.groupby(['USER_ID_hash'])
uchar = grouped.mean()
uchar.DISCOUNT_PRICE = 1
uchar.PRICE_RATE = 1
uchar.shape

In [ ]:
del train2['COUPON_ID_hash']
grouped2 = train2.groupby(['USER_ID_hash'])
uchar2 = grouped2.mean()
uchar2.DISCOUNT_PRICE = 1
uchar2.PRICE_RATE = 1
uchar2.shape

In [ ]:
uchar[:3]

In [ ]:
uchar2[:3]

In [ ]:
W = np.diag(
    ([2.21] * 1) +
    ([-0.2] * 1) +
    ([0] * 9) +
    ([2.05] * 13) +
    ([0.51] * 9) +
    ([1.21] * 47) +
    ([5.75] * 55))
#W = np.diag([1] * 135)
W.shape

In [ ]:
DataFrame(W)[:3]

In [ ]:
test_mx = test.copy()
del test_mx['COUPON_ID_hash']
test_mx.shape

In [ ]:
r1 = np.dot((uchar.loc[:,:]).as_matrix(), W)
r1.shape

In [ ]:
r12 = np.dot((uchar2.loc[:,:]).as_matrix(), W)
r12.shape

In [ ]:
DataFrame(r1)[:3]

In [ ]:
DataFrame(r12)[:3]

In [ ]:
r2 = np.dot(r1, test_mx.as_matrix().T)
r2.shape

In [ ]:
r22 = np.dot(r12, test_mx.as_matrix().T)
r22.shape

In [ ]:
DataFrame(r2)[:3]

In [ ]:
DataFrame(r22)[:3]

In [ ]:
score = DataFrame(r2, index=uchar.index, columns=test.COUPON_ID_hash)
score[:3]

In [ ]:
score.shape

In [ ]:
score2 = DataFrame(r22, index=uchar2.index, columns=test.COUPON_ID_hash)
score2 = 0.25 * score2
score2[:3]

In [ ]:
score2.shape

In [ ]:
score.info()

In [ ]:
score2.info()

In [ ]:
score3 = pd.concat([score, score2])
score3.info()

In [ ]:
score3[:3]

In [ ]:
score3['c'] = score3.index

In [ ]:
grouped3 = score3.groupby(['c'])
score4 = grouped3.sum()

In [ ]:
score4.info()

In [ ]:
score4[:3]

In [ ]:
def get_recommended_coupons(row):
    row.sort(ascending=False)
    ret = " ".join(row.index[:10])
    return ret
rec = score4[:].apply(lambda row: get_recommended_coupons(row), axis=1)

In [ ]:
recf = DataFrame({'PURCHASED_COUPONS': rec})
recf[:3]
recf['USER_ID_hash'] = recf.index

In [ ]:
ssub = pd.read_csv('data/sample_submission.csv')
del ssub['PURCHASED_COUPONS']
ssub[:3]

In [ ]:
sub = pd.merge(ssub, recf, how='outer', on='USER_ID_hash')
sub[:3]

In [ ]:
sub.info()

In [ ]:
sub = sub.fillna("")
sub.info()

In [ ]:
sub.to_csv('submission/cos_sim_py_06.csv', index=False)